# GBRAS-Net

## Libraries

In [1]:
!pip install opencv-python

In [2]:
from scipy import misc, ndimage, signal
from sklearn.model_selection  import train_test_split
import numpy
import numpy as np
import random
import ntpath
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from tensorflow.keras import optimizers 
from tensorflow.keras import regularizers
import tensorflow as tf
import cv2
from tensorflow.keras import backend as K
from time import time
import time as tm
import datetime
from operator import itemgetter
import glob
from skimage.util.shape import view_as_blocks
from tensorflow.keras.utils import to_categorical

## GPU

In [22]:
# !nvidia-smi
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")
#!kill PIDnumber

No GPU found


## 30 SRM filters for preprocessing and the activation function

In [4]:
################################################## 30 SRM FILTERS
srm_weights = np.load('SRM_Kernels1.npy') 
biasSRM=numpy.ones(30)
print (srm_weights.shape)
################################################## TLU ACTIVATION FUNCTION
T3 = 3;
def Tanh3(x):
    tanh3 = K.tanh(x)*T3
    return tanh3
##################################################

(5, 5, 1, 30)


## GBRAS-Net architecture

In [5]:
def GBRAS_Net():
    tf.keras.backend.clear_session()
    #Inputs
    inputs = tf.keras.Input(shape=(512,512,1), name="input_1")
    #Layer 1
    layers = tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,biasSRM], strides=(1,1), padding='same', trainable=False, activation=Tanh3, use_bias=True)(inputs)
    layers1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 2
    layers = tf.keras.layers.DepthwiseConv2D(1)(layers1)
    layers = tf.keras.layers.SeparableConv2D(30,(3,3), padding='same', activation="elu",depth_multiplier=3)(layers) 
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 3
    layers = tf.keras.layers.DepthwiseConv2D(1)(layers)
    layers = tf.keras.layers.SeparableConv2D(30,(3,3), padding='same', activation="elu",depth_multiplier=3)(layers) 
    layers2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    skip1 =   tf.keras.layers.Add()([layers1, layers2])
    #Layer 4
    layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="elu", padding='same', kernel_initializer='glorot_uniform')(skip1) 
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 5
    layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="elu", padding='same', kernel_initializer='glorot_uniform')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 6
    layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)
    #Layer 7
    layers = tf.keras.layers.Conv2D(60, (3,3), strides=(1,1), activation="elu", padding='same', kernel_initializer='glorot_uniform')(layers) 
    layers3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 8
    layers = tf.keras.layers.DepthwiseConv2D(1)(layers3)
    layers = tf.keras.layers.SeparableConv2D(60,(3,3), padding='same', activation="elu",depth_multiplier=3)(layers) 
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 9
    layers = tf.keras.layers.DepthwiseConv2D(1)(layers)
    layers = tf.keras.layers.SeparableConv2D(60,(3,3), padding='same', activation="elu",depth_multiplier=3)(layers) 
    layers4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    skip2 =   tf.keras.layers.Add()([layers3, layers4])
    #Layer 10
    layers = tf.keras.layers.Conv2D(60, (3,3), strides=(1,1), activation="elu", padding='same', kernel_initializer='glorot_uniform')(skip2) 
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 11
    layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)
    #Layer 12
    layers = tf.keras.layers.Conv2D(60, (3,3), strides=(1,1), activation="elu", padding='same', kernel_initializer='glorot_uniform')(layers) 
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 13
    layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)
    #Layer 14
    layers = tf.keras.layers.Conv2D(60, (3,3), strides=(1,1), activation="elu", padding='same', kernel_initializer='glorot_uniform')(layers) 
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 15
    layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)
    #Layer 16
    layers = tf.keras.layers.Conv2D(30, (1,1), strides=(1,1), activation="elu", padding='same', kernel_initializer='glorot_uniform')(layers) 
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 17
    layers = tf.keras.layers.Conv2D(2, (1,1), strides=(1,1), activation="elu", padding='same', kernel_initializer='glorot_uniform')(layers) 
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 18
    layers = tf.keras.layers.GlobalAveragePooling2D(data_format="channels_last")(layers)
    #Layer 19
    predictions = tf.keras.layers.Softmax(axis=1)(layers)
    #Model generation
    model = tf.keras.Model(inputs = inputs, outputs=predictions)
    #Optimizer
    optimizer=tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    print ("Model GBRAS-Net Generated")
    #Model compilation
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


## Paths to save trained models epoch by epoch and final accuracy and loss curves

In [6]:
path_log_base = './logs'
path_img_base = './images'
if not os.path.exists(path_log_base):
    os.makedirs(path_log_base)
if not os.path.exists(path_img_base):
    os.makedirs(path_img_base)

## Defining different functions to work with the architecture

In [19]:
def train(model, X_train, y_train, X_valid, y_valid, X_test, y_test, batch_size, epochs, model_name=""):
    print('Start training of model...')
    start_time = tm.time()
    log_dir=path_log_base+"/"+model_name+"_"+str(datetime.datetime.now().isoformat()[:19].replace("T", "_").replace(":","-"))
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)
    filepath = log_dir+"/saved-model-{epoch:03d}-{val_accuracy:.4f}.hdf5"
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=False, mode='max')
    model.reset_states()
    
    global lossTEST
    global accuracyTEST
    global lossTRAIN
    global accuracyTRAIN
    global lossVALID
    global accuracyVALID
    print('Start evaluation on clean model...')
    print('\t TEST')
    lossTEST,accuracyTEST   = model.evaluate(X_test, y_test,verbose=1)
    lossTRAIN,accuracyTRAIN = model.evaluate(X_train, y_train,verbose=1)
    lossVALID,accuracyVALID = model.evaluate(X_valid, y_valid,verbose=1)

    global history
    global model_Name
    global log_Dir
    model_Name = model_name
    log_Dir = log_dir
    print("Starting the training...")
    history=model.fit(X_train, y_train, epochs=epochs, 
                      callbacks=[tensorboard,checkpoint], 
                      batch_size=batch_size,validation_data=(X_valid, y_valid),verbose=2)
    
    metrics = model.evaluate(X_test, y_test, verbose=0)
     
    TIME = tm.time() - start_time
    print("Time "+model_name+" = %s [seconds]" % TIME)
    
    print("\n")
    print(log_dir)
    return {k:v for k,v in zip (model.metrics_names, metrics)}

In [8]:
def Final_Results_Test(PATH_trained_models):
    global AccTest
    global LossTest
    AccTest = []
    LossTest= [] 
    B_accuracy = 0 #B --> Best
    for filename in sorted(os.listdir(PATH_trained_models)):
        if filename != ('train') and filename != ('validation'):
            print(filename)
            model = tf.keras.models.load_model(PATH_trained_models+'/'+filename, custom_objects={'Tanh3':Tanh3})
            loss,accuracy = model.evaluate(X_test, y_test,verbose=0)
            print(f'Loss={loss:.4f} y Accuracy={accuracy:0.4f}'+'\n')
            BandAccTest  = accuracy
            BandLossTest = loss
            AccTest.append(BandAccTest)    
            LossTest.append(BandLossTest)  
            
            if accuracy > B_accuracy:
                B_accuracy = accuracy
                B_loss = loss
                B_name = filename
    
    print("\n\nBest")
    print(B_name)
    print(f'Loss={B_loss:.4f} y Accuracy={B_accuracy:0.4f}'+'\n')

In [9]:
def graphics(history, AccTest, LossTest, log_Dir, model_Name, lossTEST, lossTRAIN, lossVALID, accuracyTEST, accuracyTRAIN, accuracyVALID):
    numbers=AccTest
    numbers_sort = sorted(enumerate(numbers), key=itemgetter(1),  reverse=True)
    for i in range(int(len(numbers)*(0.05))): #5% total epochs
        index, value = numbers_sort[i]
        print("Test Accuracy {}, epoch:{}\n".format(value, index+1))
    
    print("")
    
    numbers=history.history['accuracy']
    numbers_sort = sorted(enumerate(numbers), key=itemgetter(1),  reverse=True)
    for i in range(int(len(numbers)*(0.05))): #5% total epochs
        index, value = numbers_sort[i]
        print("Train Accuracy {}, epoch:{}\n".format(value, index+1))
    
    print("")
    
    numbers=history.history['val_accuracy']
    numbers_sort = sorted(enumerate(numbers), key=itemgetter(1),  reverse=True)
    for i in range(int(len(numbers)*(0.05))): #5% total epochs
        index, value = numbers_sort[i]
        print("Validation Accuracy {}, epoch:{}\n".format(value, index+1))

    with plt.style.context('seaborn-white'):
        plt.figure(figsize=(10, 10))
        plt.plot(np.concatenate([np.array([accuracyTRAIN]),np.array(history.history['accuracy'])],axis=0))
        plt.plot(np.concatenate([np.array([accuracyVALID]),np.array(history.history['val_accuracy'])],axis=0))
        plt.plot(np.concatenate([np.array([accuracyTEST]),np.array(AccTest)],axis=0)) #Test
        plt.title('Accuracy Vs Epoch')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation', 'Test'], loc='upper left')
        plt.grid('on')
        plt.savefig(path_img_base+'/Accuracy_GBRAS_Net_'+model_Name+'.eps', format='eps')
        plt.savefig(path_img_base+'/Accuracy_GBRAS_Net_'+model_Name+'.svg', format='svg')
        plt.savefig(path_img_base+'/Accuracy_GBRAS_Net_'+model_Name+'.pdf', format='pdf')     
        plt.show()
        
        plt.figure(figsize=(10, 10))
        plt.plot(np.concatenate([np.array([lossTRAIN]),np.array(history.history['loss'])],axis=0))
        plt.plot(np.concatenate([np.array([lossVALID]),np.array(history.history['val_loss'])],axis=0))
        plt.plot(np.concatenate([np.array([lossTEST]),np.array(LossTest)],axis=0)) #Test
        plt.title('Loss Vs Epoch')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation', 'Test'], loc='upper left')
        plt.grid('on')
        plt.savefig(path_img_base+'/Loss_GBRAS_Net_'+model_Name+'.eps', format='eps')
        plt.savefig(path_img_base+'/Loss_GBRAS_Net_'+model_Name+'.svg', format='svg')
        plt.savefig(path_img_base+'/Loss_GBRAS_Net_'+model_Name+'.pdf', format='pdf') 
        plt.show() 

# Working with BOSSbase 1.01

## Loading BOSSbase 1.01 dataset for S-UNIWARD 0.4bpp

In [14]:
n=512
from tqdm.notebook import tqdm # For displaying progress
from math import ceil
def load_images(path_pattern):
    files=glob.glob(path_pattern)
    X=[]
    for f in tqdm(sorted(files), f'Loading {path_pattern}'):
        I = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        patches = view_as_blocks(I, (n, n))
        for i in range(patches.shape[0]):
            for j in range(patches.shape[1]):
                X.append( [ patches[i,j] ] )
    X=numpy.array(X)
    return X

pathc = r'E:\ml\notebook_train\dataset\VISION\cover'
paths = r'E:\ml\notebook_train\dataset\VISION\HUGO\40'

Xc_ = load_images(pathc+'\*.pgm') ##COVER IMAGES
Xs_ = load_images(paths+'\*.pgm') ##STEGO IMAGES

X_  = (numpy.vstack((Xc_, Xs_)))
Xt_ = (numpy.hstack(([0]*len(Xc_), [1]*len(Xs_))))
Xt_ = to_categorical(Xt_, 2)
print(Xt_.shape)
X_  = np.rollaxis(X_,1,4)  #channel axis shifted to last axis

print("Total image data and labels",X_.shape,Xt_.shape)
#Cover hasta las 10000 ##Train hasta las 4000 ##Valid hasta de las 4000 a las 5000 ##Test de las 5000 a las 10000

ds_len = len(Xc_)
train_split = 0.4
valid_split = 0.1
test_split = 0.5

train_idx = ceil(ds_len * train_split)
valid_idx = ceil(ds_len * valid_split)
test_idx = ceil(ds_len * test_split)

train_bound_end = train_idx
valid_bound_start = train_bound_end
valid_bound_end = train_idx + valid_idx
test_bound_start = valid_bound_end
test_bound_end = valid_bound_end + test_idx

print('DS len: {0}, train: 0:{1}, valid: {1}:{2}, test: {2}:{3}'.format(ds_len, train_bound_end, valid_bound_end, test_bound_end))

X_train = np.concatenate([X_[0:train_bound_end],X_[ds_len:ds_len+train_bound_end]],axis=0)
X_valid = np.concatenate([X_[valid_bound_start:valid_bound_end],X_[ds_len+valid_bound_start:ds_len+valid_bound_end]],axis=0)
X_test  = np.concatenate([X_[test_bound_start:test_bound_end],X_[ds_len+test_bound_start:ds_len+test_bound_end]],axis=0)
y_train = np.concatenate([Xt_[0:train_bound_end],Xt_[ds_len:ds_len+train_bound_end]],axis=0)
y_valid = np.concatenate([Xt_[valid_bound_start:valid_bound_end],Xt_[ds_len+valid_bound_start:ds_len+valid_bound_end]],axis=0)
y_test  = np.concatenate([Xt_[test_bound_start:test_bound_end],Xt_[ds_len+test_bound_start:ds_len+test_bound_end]],axis=0)
#Controled randomized data for training
X_dat0, X_dat1, y_dat0, y_dat1 = train_test_split(X_train, y_train, test_size=0.50, random_state=64) 
X_train = np.concatenate([X_dat0,X_dat1],axis=0) 
y_train = np.concatenate([y_dat0,y_dat1],axis=0) 
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)
print(y_test.shape)

Loading E:\ml\notebook_train\dataset\VISION\cover\*.pgm:   0%|          | 0/9976 [00:00<?, ?it/s]

Loading E:\ml\notebook_train\dataset\VISION\HUGO\40\*.pgm:   0%|          | 0/9976 [00:00<?, ?it/s]

(19952, 2)
Total image data and labels (19952, 512, 512, 1) (19952, 2)
DS len: 9976, train: 0:3991, valid: 3991:4989, test: 4989:9977
(7982, 512, 512, 1)
(7982, 2)
(1996, 512, 512, 1)
(1996, 2)
(9975, 512, 512, 1)
(9975, 2)


## Load GBRAS-Net architecture, and show the layers

In [15]:
model = GBRAS_Net()
model.summary()

Model GBRAS-Net Generated
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 30) 780         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 512, 512, 30) 90          conv2d[0][0]                     
__________________________________________________________________________________________________
depthwise_conv2d (DepthwiseConv (None, 512, 512, 30) 60          batch_normalization[0][0]        
_____________________________________________________________

In [16]:
#tf.keras.utils.plot_model(model, show_shapes=True) 

## Train the GBRAS-Net architecture

In [21]:
with tf.device('/device:GPU:0'):
    train(model, X_train, y_train, X_valid, y_valid, X_test, y_test, batch_size=10, epochs=100, model_name="model_GBRAS-Net_ALASKA_HUGO_04bpp") 


Start training of model...
Start evaluation on clean model...
	 TEST
  1/312 [..............................] - ETA: 0s - loss: 1.2786 - accuracy: 0.0000e+00

KeyboardInterrupt: 

## Show the performance on Test data

In [ ]:
Final_Results_Test(log_Dir) 

## Shows the best 5% of all trained and saved models in the test data set and shows accuracy and loss curves

In [ ]:
# GRAPHICS OF THE THREE (TRAIN, TEST AND VALIDATION) CURVES
graphics(history, AccTest, LossTest, log_Dir, model_Name, lossTEST, lossTRAIN, lossVALID, accuracyTEST, accuracyTRAIN, accuracyVALID)


In [ ]:
print(log_Dir)

# NOTE for 0.2bpp experiments

### To work with 0.2bpp, load a model where CNN has already converged to a certain extent. (in about 25 epochs of a model trained with 0.4bpp or a model with 60% accuracy). You can see the behavior and accuracy that the architecture reaches with the steganographic algorithm 0.2bpp.

path_log = 'PATH' #put the path where the models have been saved in the training process
model_name = '/NAME.hdf5' #put the name of the selected model with an 60% accuracy for example
model = tf.keras.models.load_model(path_log+model_name, custom_objects={'Tanh3':Tanh3}, compile=True)
loss,accuracy = model.evaluate(X_test,y_test,verbose=0)
print(f'Loss={loss:.4f} and Accuracy={accuracy:0.4f}') 


# Working with BOSSbase 1.01 + BOWS2

### In the paper experiments, we took BOSSbase 1.01 and BOWS2 only with 0.2bpp.

## 1) Load the best model obtained with BOSSbase 1.01

path_log = 'PATH' #put the path where the models have been saved in the training process
model_name = '/NAME_OF_THE_BEST_MODEL.hdf5' #put the name of the BEST MODEL FOR BOSSBASE 1.01
model = tf.keras.models.load_model(path_log+model_name, custom_objects={'Tanh3':Tanh3}, compile=True)
loss,accuracy = model.evaluate(X_test,y_test,verbose=0) #Test BOSSbase 1.01
print(f'Loss={loss:.4f} and Accuracy={accuracy:0.4f}') 


## 2) Load the BOWS2 dataset

n=256
def load_images(path_pattern):
    files=glob.glob(path_pattern)
    X=[]
    for f in sorted(files):
        I = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        patches = view_as_blocks(I, (n, n))
        for i in range(patches.shape[0]):
            for j in range(patches.shape[1]):
                X.append( [ patches[i,j] ] )
    X=numpy.array(X)
    return X

pathc = './DATABASES/BOWS2'
paths = './DATABASES/BOWS2/S-UNIWARD/0.2bpp'

Xc_ = load_images(pathc+'/cover/*.pgm') ##COVER IMAGES
Xs_ = load_images(paths+'/stego/*.pgm') ##STEGO IMAGES
X_  = (numpy.vstack((Xc_, Xs_)))
X_  = np.rollaxis(X_,1,4) 
Xt_ = (numpy.hstack(([0]*len(Xc_), [1]*len(Xs_))))
Xt_ = np_utils.to_categorical(Xt_, 2)

print(X_.shape) # 10000 image pairs from BOWS2
print(Xt_.shape)

print(X_train.shape) # 4000 image pairs from BOSSbase 1.01
print(y_train.shape)

X_train = np.concatenate([X_train,X_],axis=0) #Controled randomized data for training, joining 4000 image pairs from BOSSbase 1.01 and 10000 image pairs from BOWS2
y_train = np.concatenate([y_train,Xt_],axis=0)

X_dat0, X_dat1, y_dat0, y_dat1 = train_test_split(X_train, y_train, test_size=0.50, random_state=64) 
X_train = np.concatenate([X_dat0,X_dat1],axis=0) 
y_train = np.concatenate([y_dat0,y_dat1],axis=0) 

print(X_train.shape) #14000 image pairs, 4000 from BOSSbase 1.01 and 10000 from BOWS2
print(y_train.shape)

## 3) Train the architecture BOSSbase 1.01 and BOWS2

train(model, X_train, y_train, X_valid, y_valid, X_test, y_test, batch_size=32, epochs=100, model_name="model_GBRAS-Net_S_UNIWARD_04bpp_BOSSbase_1.01_and_BOWS2")


## 4) Show the performance on Test data

Final_Results_Test(log_Dir) 

## 5) Shows the best 5% of all trained and saved models in the test data set and shows accuracy and loss curves. (TRAIN, TEST AND VALIDATION CURVES)

graphics(history, AccTest, LossTest, log_Dir, model_Name, lossTEST, lossTRAIN, lossVALID, accuracyTEST, accuracyTRAIN, accuracyVALID)
